In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import random
from dataclasses import dataclass

@dataclass
class TrainCfg:
    batch_size: int = 128
    max_epochs: int = 150
    lr: float = 3e-4
    device: str = "cuda" if torch.cuda.is_available() else "cpu"


CFG = TrainCfg()

In [2]:
class CardioRiskNet(nn.Module):
    def __init__(self, input_size):
        super().__init__()
        self.fc1 = nn.Linear(input_size, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 1)
        self.dropout = nn.Dropout(0.3)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        return torch.sigmoid(self.fc3(x))


In [ ]:
import torch
from torch.utils.data import Dataset
import random

def sample_regime():
    regimes = [
        "healthy",
        "lifestyle",
        "hypertensive",
        "cardiopulmonary",
        "high_risk"
    ]
    probs = [0.30, 0.25, 0.20, 0.15, 0.10]
    return random.choices(regimes, probs)[0]

def sample_patient_by_regime(regime):
    if regime == "healthy":
        age = random.uniform(20, 40)
        bmi = random.uniform(18.5, 24)
        sys_bp = random.uniform(90, 120)
        dia_bp = random.uniform(60, 80)
        heart_rate = random.uniform(55, 80)
        spo2 = random.uniform(97, 100)

        exercise = random.uniform(0.7, 1.0)
        diet = random.uniform(0.7, 1.0)
        sleep = random.uniform(0.7, 1.0)

        smoking = random.uniform(0, 10)
        alcohol = random.uniform(0, 7)
        ecg_prob = random.uniform(0.0, 0.2)

    elif regime == "lifestyle":
        age = random.uniform(25, 55)
        bmi = random.uniform(24, 30)
        sys_bp = random.uniform(110, 140)
        dia_bp = random.uniform(70, 90)
        heart_rate = random.uniform(65, 95)
        spo2 = random.uniform(95, 99)

        exercise = random.uniform(0.2, 0.6)
        diet = random.uniform(0.2, 0.6)
        sleep = random.uniform(0.3, 0.6)

        smoking = random.uniform(20, 80)
        alcohol = random.uniform(5, 20)
        ecg_prob = random.uniform(0.1, 0.4)

    elif regime == "hypertensive":
        age = random.uniform(45, 75)
        bmi = random.uniform(26, 35)
        sys_bp = random.uniform(140, 180)
        dia_bp = random.uniform(85, 120)
        heart_rate = random.uniform(70, 105)
        spo2 = random.uniform(94, 98)

        exercise = random.uniform(0.2, 0.5)
        diet = random.uniform(0.3, 0.6)
        sleep = random.uniform(0.4, 0.7)

        smoking = random.uniform(10, 60)
        alcohol = random.uniform(5, 15)
        ecg_prob = random.uniform(0.3, 0.6)

    elif regime == "cardiopulmonary":
        age = random.uniform(50, 80)
        bmi = random.uniform(22, 32)
        sys_bp = random.uniform(120, 170)
        dia_bp = random.uniform(70, 100)
        heart_rate = random.uniform(80, 110)
        spo2 = random.uniform(88, 95)

        exercise = random.uniform(0.1, 0.4)
        diet = random.uniform(0.3, 0.6)
        sleep = random.uniform(0.3, 0.6)

        smoking = random.uniform(30, 100)
        alcohol = random.uniform(5, 20)
        ecg_prob = random.uniform(0.4, 0.8)

    else:  # high_risk
        age = random.uniform(60, 85)
        bmi = random.uniform(28, 40)
        sys_bp = random.uniform(150, 200)
        dia_bp = random.uniform(90, 130)
        heart_rate = random.uniform(90, 130)
        spo2 = random.uniform(85, 92)

        exercise = random.uniform(0.0, 0.3)
        diet = random.uniform(0.1, 0.4)
        sleep = random.uniform(0.2, 0.5)

        smoking = random.uniform(60, 140)
        alcohol = random.uniform(10, 30)
        ecg_prob = random.uniform(0.6, 1.0)

    sex = random.choice([0, 1])

    return {
        "age": age,
        "sex": sex,
        "bmi": bmi,
        "sys_bp": sys_bp,
        "dia_bp": dia_bp,
        "heart_rate": heart_rate,
        "spo2": spo2,
        "exercise": exercise,
        "diet": diet,
        "sleep": sleep,
        "smoking": smoking,
        "alcohol": alcohol,
        "ecg_prob": ecg_prob
    }

class SyntheticPatientDataset(Dataset):
    def __init__(self, num_samples=2000, cfg=CFG):
        self.features = []
        self.targets = []

        for _ in range(num_samples):
            regime = sample_regime()
            p = sample_patient_by_regime(regime)

            age_norm = (p["age"] - 20) / 65
            bmi_norm = (p["bmi"] - 18.5) / 21.5
            sys_bp_norm = (p["sys_bp"] - 90) / 110
            dia_bp_norm = (p["dia_bp"] - 60) / 70
            smoking_norm = p["smoking"] / 140
            alcohol_norm = p["alcohol"] / 30
            hr_norm = min(abs(p["heart_rate"] - 75) / 40, 1.0)
            spo2_norm = min((100 - p["spo2"]) / 15, 1.0)

            feat = [
                p["ecg_prob"],
                p["exercise"],
                p["diet"],
                p["sleep"],
                smoking_norm,
                alcohol_norm,
                age_norm,
                p["sex"],
                bmi_norm,
                sys_bp_norm,
                dia_bp_norm,
                hr_norm,
                spo2_norm
            ]

            self.features.append(torch.tensor(feat, dtype=torch.float32))

            baseline_risk = (
                0.30 * age_norm +
                0.10 * p["sex"] +
                0.15 * bmi_norm
            )

            vitals_risk = (
                0.20 * sys_bp_norm +
                0.15 * dia_bp_norm +
                0.15 * hr_norm +
                0.25 * spo2_norm
            )

            lifestyle_risk = (
                0.20 * smoking_norm +
                0.05 * alcohol_norm +
                0.15 * (1 - p["exercise"]) +
                0.10 * (1 - p["diet"]) +
                0.10 * (1 - p["sleep"])
            )

            ecg_risk = 0.35 * p["ecg_prob"]

            risk = (
                0.30 * baseline_risk +
                0.35 * vitals_risk +
                0.20 * lifestyle_risk +
                ecg_risk
            )

            risk = min(max(risk, 0.0), 1.0)
            self.targets.append(torch.tensor(risk, dtype=torch.float32))

        self.features = torch.stack(self.features).to(cfg.device)
        self.targets = torch.stack(self.targets).to(cfg.device)

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        return self.features[idx], self.targets[idx]

In [4]:
def train_model(model, dataset, cfg=CFG):
    loader = DataLoader(dataset, batch_size=cfg.batch_size, shuffle=True)
    optimizer = torch.optim.Adam(model.parameters(), lr=CFG.lr)
    criterion = nn.MSELoss()

    for epoch in range(cfg.max_epochs):
        total_loss = 0
        model.train()
        for X, y in loader:
            optimizer.zero_grad()
            pred = model(X).squeeze()
            loss = criterion(pred, y)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        print(f"Epoch {epoch+1}/{cfg.max_epochs}, Loss: {total_loss/len(loader):.4f}")
    return model



In [7]:
ds = SyntheticPatientDataset(10000)


model = CardioRiskNet(input_size=13).to(CFG.device)

model = train_model(model, ds)



Epoch 1/150, Loss: 0.0704
Epoch 2/150, Loss: 0.0377
Epoch 3/150, Loss: 0.0141
Epoch 4/150, Loss: 0.0064
Epoch 5/150, Loss: 0.0048
Epoch 6/150, Loss: 0.0042
Epoch 7/150, Loss: 0.0039
Epoch 8/150, Loss: 0.0037
Epoch 9/150, Loss: 0.0033
Epoch 10/150, Loss: 0.0033
Epoch 11/150, Loss: 0.0031
Epoch 12/150, Loss: 0.0029
Epoch 13/150, Loss: 0.0028
Epoch 14/150, Loss: 0.0026
Epoch 15/150, Loss: 0.0026
Epoch 16/150, Loss: 0.0025
Epoch 17/150, Loss: 0.0024
Epoch 18/150, Loss: 0.0023
Epoch 19/150, Loss: 0.0023
Epoch 20/150, Loss: 0.0023
Epoch 21/150, Loss: 0.0023
Epoch 22/150, Loss: 0.0022
Epoch 23/150, Loss: 0.0022
Epoch 24/150, Loss: 0.0021
Epoch 25/150, Loss: 0.0020
Epoch 26/150, Loss: 0.0020
Epoch 27/150, Loss: 0.0020
Epoch 28/150, Loss: 0.0019
Epoch 29/150, Loss: 0.0020
Epoch 30/150, Loss: 0.0019
Epoch 31/150, Loss: 0.0019
Epoch 32/150, Loss: 0.0019
Epoch 33/150, Loss: 0.0019
Epoch 34/150, Loss: 0.0018
Epoch 35/150, Loss: 0.0019
Epoch 36/150, Loss: 0.0018
Epoch 37/150, Loss: 0.0018
Epoch 38/1

In [8]:
model.eval()

torch.save(model.state_dict(), "cardiorisknet_model.pt")

In [ ]:
age, smoking_per_week, alcohol_per_week = 20, 0, 0
height_cm, weight_kg = 175, 65
sys_bp, dia_bp = 110, 70

age_norm = (age - 20) / 60
smoking_norm = smoking_per_week / 140
alcohol_norm = alcohol_per_week / 30

height_m = height_cm / 100.0
bmi = weight_kg / (height_m ** 2)
bmi_norm = (bmi - 18.5) / 16.5
bmi_norm = max(0.0, min(bmi_norm, 1.0))

sys_bp_norm = (sys_bp - 90) / 90
dia_bp_norm = (dia_bp - 60) / 60


In [ ]:
model(torch.tensor([0.13, 1, 1, 1, smoking_norm, alcohol_norm, age_norm, 1, bmi_norm, sys_bp_norm, dia_bp_norm]))

tensor([0.0224], grad_fn=<SigmoidBackward0>)